| Feature                               | **FAISS**                                                    | **Chroma**                                                          |
| ------------------------------------- | ------------------------------------------------------------ | ------------------------------------------------------------------- |
| **Type**                              | In-memory **vector index library** (by Facebook AI)          | **Vector database** with persistence and metadata management        |
| **Stores Data**                       | In RAM (unless you manually save/load)                       | On disk (auto-persistent)                                           |
| **Supports Collections / Namespaces** | ❌ No concept of "collection name" — only one index at a time | ✅ Has **collections**, each acts like a separate table or namespace |
| **Use Case**                          | Fast, lightweight embedding search inside your app           | Persistent and organized vector database for long-term storage      |


In [6]:
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [7]:
# Sample documents
docs = [
    Document(page_content="LangChain makes it easy to work with LLMs."),
    Document(page_content="LangChain is used to build LLM based applications."),
    Document(page_content="Chroma is used to store and search document embeddings."),
    Document(page_content="Embeddings are vector representations of text."),
    Document(page_content="MMR helps you get diverse results when doing similarity search."),
    Document(page_content="LangChain supports Chroma, FAISS, Pinecone, and more."),
]

In [8]:
docs

[Document(metadata={}, page_content='LangChain makes it easy to work with LLMs.'),
 Document(metadata={}, page_content='LangChain is used to build LLM based applications.'),
 Document(metadata={}, page_content='Chroma is used to store and search document embeddings.'),
 Document(metadata={}, page_content='Embeddings are vector representations of text.'),
 Document(metadata={}, page_content='MMR helps you get diverse results when doing similarity search.'),
 Document(metadata={}, page_content='LangChain supports Chroma, FAISS, Pinecone, and more.')]

In [9]:
embedding_model=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [11]:
vectorstore=FAISS.from_documents(
    documents=docs, 
    embedding=embedding_model
)

In [13]:
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 1}  # k = top results, lambda_mult = relevance-diversity balance
)

query='what is langchain?'
results=retriever.invoke(query)

for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain supports Chroma, FAISS, Pinecone, and more.

--- Result 2 ---
LangChain is used to build LLM based applications.

--- Result 3 ---
LangChain makes it easy to work with LLMs.


In [14]:
# Enable MMR in the retriever
retriever = vectorstore.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

query='what is langchain?'
results=retriever.invoke(query)

for i, doc in enumerate(results):
    print(f"\n--- Result {i+1} ---")
    print(doc.page_content)


--- Result 1 ---
LangChain supports Chroma, FAISS, Pinecone, and more.

--- Result 2 ---
LangChain is used to build LLM based applications.

--- Result 3 ---
Embeddings are vector representations of text.


In [15]:
# lambda_mult ===> 0(most diverse)------------------------------>1
                                # reduce divers result